In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


{'dag_runs': [{'conf': {},
   'dag_id': 'DMS_MASTER_CUSTOMER',
   'dag_run_id': 'scheduled__2022-07-12T18:00:00+00:00',
   'end_date': '2022-07-13T18:00:22.315811+00:00',
   'execution_date': '2022-07-12T18:00:00+00:00',
   'external_trigger': False,
   'start_date': '2022-07-13T18:00:00.964662+00:00',
   'state': 'success'}],
 'total_entries': 8}

In [ ]:
### BoilerPlate
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='TonKhoAdHoc'
prefix='SC_'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 4, 2, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '@once',
          tags=[prefix+name, 'adhoc']
)

csv_path = '/usr/local/airflow/plugins'+'/'
### END BOILERPLATE

In [6]:
from utils.df_handle import *


In [4]:
x = 1 if datetime.now().weekday() == 0 else 1
datenow = datetime.now().strftime("%Y%m%d")
datenow_add1 = (datetime.now() + timedelta(days=x)).strftime("%Y%m%d")
datenow_mns1 = (datetime.now() - timedelta(days=x)).strftime("%Y%m%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")

In [7]:
csv_path = ''

In [8]:
df5 = pd.read_csv(csv_path+f"{datenow_mns1}_DF5.csv")
df5.chinhanh.fillna("NA", inplace=True)

In [12]:
# df5.shape

In [10]:
assert checkdup(df5,1,['masanpham','chinhanh']).sum() == 0, "MSP & Chi Nhanh khong duoc trung"

In [13]:
# Map new data 13/01
HDDURL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTlHJ6SB5QNdaHdEaJlRnU7nKcLI2Haj6YlebHtqMvJ-GKsmAZWRvWa5j5dKBY8INF2vSd1fSJlTrXs/pub?gid=0&single=true&output=csv'
_df = pd.read_csv(HDDURL, dayfirst=True, parse_dates=['Ngay(dd/mm/yyyy)'])
lst = ['ngay','masanpham', 'tensanpham', 'soluonghdd1', 'chinhanh', 'solo', 'exdate', 'soluonghdd']
# lst = ['ngay','masanpham', 'tensanpham', 'chinhanh','soluonghdd1','soluonghdd']
_df.columns = lst
_df = _df[_df.masanpham.notna()]
_df.chinhanh.fillna("NA", inplace=True)
drop_cols(_df,['ngay','soluonghdd1', 'solo', 'exdate'])
_df = pivot(_df, ['masanpham', 'chinhanh'], {'soluonghdd':np.sum})

# Update 11/02/2022
df5_dict = df5[['masanpham','tensanpham','donvi']].copy()
df5_dict.drop_duplicates(inplace=True)
_df = _df.merge(df5_dict, how='left', on='masanpham')
# Update 11/02/2022
df5_dict = df5[['chinhanh','songaynhan']].copy()
df5_dict.drop_duplicates(inplace=True)
_df = _df.merge(df5_dict, how='left', on='chinhanh')

_df['tonao'] = 0
_df['toncn'] = 0
_df['tonhcm'] = 0
_df['tonmerap'] = 0
_df['tonvime'] = 0
_df['toncn_huy'] = 0
_df['tonhcm_huy'] = 0

_df.columns = ['masanpham','chinhanh','tonhangdiduong','tensanpham','donvi','songaynhan','tonao','toncn','tonhcm','tonmerap','tonvime', 'toncn_huy', 'tonhcm_huy']
_df = _df[['masanpham', 'tensanpham', 'donvi', 'chinhanh', 'songaynhan', 'tonao', 'tonhangdiduong', 'toncn', 'tonhcm', 'tonmerap', 'tonvime', 'toncn_huy', 'tonhcm_huy']]
df5 = union_all([df5, _df])
df5 = pivot(df5, ['masanpham', 'tensanpham', 'donvi', 'chinhanh', 'songaynhan'], {'tonao':np.sum, 'tonhangdiduong':np.sum, 'toncn':np.sum, 'tonhcm':np.sum, 'tonmerap':np.sum, 'tonvime':np.sum, 'toncn_huy':np.sum, 'tonhcm_huy':np.sum})


assert checkdup(df5,1,['masanpham','chinhanh']).sum() == 0, "MSP & Chi Nhanh khong duoc trung"

NMURLTP = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRvmeMXGwa-2u-cCQmDRyXs__a8oLfcZk9yLyq1LupmdsvzulMVlxHublEJAKszBY-zmnl_Wm1KNnvZ/pub?gid=0&single=true&output=csv'
nmtp = pd.read_csv(NMURLTP)
# nmtp.columns = nmtp.iloc[0]
headers = nmtp.iloc[0]
nmtp  = pd.DataFrame(nmtp.values[1:], columns=headers)
nmtp.columns = cleancols(nmtp)
nmtp.columns = lower_col(nmtp)
dk1 = nmtp.ten_phanam.notna()
nmtp = nmtp[dk1].copy()
nmtp = nmtp[['ten_phanam','convert']].copy()
nmtp.convert = pd.to_numeric(nmtp.convert)
dk2 = nmtp.convert != 0
nmtp = nmtp[dk2].copy()
nmtp['datatype'] = 'TP'

# nmtp.convert.sum()

NMURLBT = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRvmeMXGwa-2u-cCQmDRyXs__a8oLfcZk9yLyq1LupmdsvzulMVlxHublEJAKszBY-zmnl_Wm1KNnvZ/pub?gid=869914713&single=true&output=csv'
nmbt = pd.read_csv(NMURLBT)
nmbt.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
nmbt = nmbt[[14,15]].copy()
nmbt.columns = ['ten_phanam', 'convert']
dk1 = nmbt.ten_phanam.notna()
nmbt = nmbt[dk1].copy()
nmbt.convert = pd.to_numeric(nmbt.convert)
dk2 = nmbt.convert != 0
nmbt = nmbt[dk2].copy()
nmbt['datatype'] = 'BT'

NMURLHH = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRvmeMXGwa-2u-cCQmDRyXs__a8oLfcZk9yLyq1LupmdsvzulMVlxHublEJAKszBY-zmnl_Wm1KNnvZ/pub?gid=1577909815&single=true&output=csv'
nmhh = pd.read_csv(NMURLHH)
nmhh.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
nmhh = nmhh[[14,15]].copy()
nmhh.columns = ['ten_phanam', 'convert']
dk1 = nmhh.ten_phanam.notna()
nmhh = nmhh[dk1].copy()
nmhh.convert = pd.to_numeric(nmhh.convert)
dk2 = nmhh.convert != 0
nmhh = nmhh[dk2].copy()
nmhh['datatype'] = 'HH'

NM = union_all([nmtp,nmbt,nmhh])
NM.columns = ['invtid','soluong','datatype']
# dfnmid = get_ps_df("SELECT nhamayid, invtid FROM d_sc_invtid WHERE nhamayid NOTNULL")
# dfnmid = pd.read_csv('datacodenm.csv')
# NM = NM.merge(dfnmid, how='left', on='nhamayid')
# assert NM.invtid.isna().sum() == 0, "NEW NM SKU FOUND"
# del(dfnmid)
# NM = NM[NM.invtid.notna()]
NM = pivot(NM, ['invtid', 'datatype'], {'soluong':np.sum})

df5_dict = df5[['masanpham','tensanpham','donvi']].copy()
df5_dict.drop_duplicates(inplace=True)

NM.columns = ['masanpham', 'datatype', 'soluong']

# ADDING Ton Theo PO & NM no PO
THEOPO_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRvmeMXGwa-2u-cCQmDRyXs__a8oLfcZk9yLyq1LupmdsvzulMVlxHublEJAKszBY-zmnl_Wm1KNnvZ/pub?gid=574959890&single=true&output=csv'
nmtheopo = pd.read_csv(THEOPO_URL)

nmtheopo.columns = cleancols(nmtheopo)
nmtheopo.columns = lower_col(nmtheopo)

nmtheopo.columns = ['mapn','dummy','nmnopo','tontheopo']

nmpo = nmtheopo[['mapn','tontheopo']].copy()
nmpo['datatype'] = 'PO'
nmpo = nmpo[['mapn', 'datatype', 'tontheopo']]
nmpo.columns = ['masanpham', 'datatype', 'soluong']

nmno = nmtheopo[['mapn','nmnopo']].copy()
nmno['datatype'] = 'NO'
nmno = nmno[['mapn', 'datatype', 'nmnopo']]
nmno.columns = ['masanpham', 'datatype', 'soluong']

NM = union_all([NM,nmpo,nmno])

# NM.to_clipboard()

NM = NM.merge(df5_dict, how='left', on='masanpham')

NM['chinhanh'] = 'NM'

dk1 = NM['datatype'] == "TP"
dk2 = NM['datatype'] == "BT"
dk3 = NM['datatype'] == "HH"
dk4 = NM['datatype'] == "PO"
dk5 = NM['datatype'] == "NO"

NM['tonnmtp'] = np.where(dk1, NM['soluong'], 0)
NM['tonnmbt'] = np.where(dk2, NM['soluong'], 0)
NM['tonnmhh'] = np.where(dk3, NM['soluong'], 0)
NM['tonnmpo'] = np.where(dk4, NM['soluong'], 0)
NM['tonnmno'] = np.where(dk5, NM['soluong'], 0)

grouplst = ['masanpham', 'tensanpham', 'donvi', 'chinhanh']
agg_dict = \
{
'tonnmtp':np.sum,
'tonnmbt':np.sum,
'tonnmhh':np.sum,
'tonnmpo':np.sum,
'tonnmno':np.sum
}

NM = pivot(NM, grouplst, agg_dict)

NM['songaynhan'] = 0
NM['tonao'] = 0
NM['tonhangdiduong'] = 0
NM['toncn'] = 0
NM['tonhcm'] = 0
NM['tonmerap'] = 0
NM['tonvime'] = 0
NM['toncn_huy'] = 0
NM['tonhcm_huy'] = 0

NM = NM[['masanpham', 'tensanpham', 'donvi', 'chinhanh', 'songaynhan', 'tonao', 'tonhangdiduong', 'toncn', 'tonhcm',
    'tonmerap', 'tonvime', 'toncn_huy', 'tonhcm_huy', 'tonnmtp', 'tonnmbt', 'tonnmhh', 'tonnmpo', 'tonnmno']]

# NM.to_clipboard()

df5['tonnmtp'] = 0
df5['tonnmbt'] = 0
df5['tonnmhh'] = 0
df5['tonnmpo'] = 0
df5['tonnmno'] = 0

# union ton NM va ton DMS
df5 = union_all([df5, NM])

# df5.head()

cur_day = datetime.now().day
cur_month = datetime.now().month
cur_year = datetime.now().year

def get_3m_ago(cur_month=datetime.now().month, cur_year=datetime.now().year):
    if cur_month - 3 < 0:
        return [cur_month + 12 - 3, cur_year -1]
    if cur_month - 3 == 0:
        return [12, cur_year -1]
    else:
        return [cur_month - 3, cur_year]


# lst = get_3m_ago()
# start_date = datetime(lst[1], lst[0], cur_day).strftime("%Y%m%d")
# # start_date
# query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI_v1] @Fromdate='{start_date}', @Todate='{datenow}'"
# SALES = get_ms_df(sql=query)
# SALES.columns = cleancols(SALES)
# SALES.columns = lower_col(SALES)

# SALES = SALES[['masanpham', 'soluong', 'makho']]
# dfsc = get_ps_df("select makho,chinhanh from d_sc_kho_chi_nhanh")
# dfsc.columns = ['makho','chinhanh_sc']
# SALES = SALES.merge(dfsc, how="left", on="makho")
# # SALES.head()
# del(dfsc)
# days = len(pd.date_range(start_date, datetime.now()))-13
# SALES = pivot(SALES, ['masanpham', "chinhanh_sc"], {"soluong":np.sum})
# SALES['avg_3m'] = round(SALES['soluong']/days,0)
# # drop_cols(SALES, 'soluong')
# SALES.columns = ['masanpham','chinhanh','soluong','avg_3m']

bsql = \
"""
with sales as 
(select masanpham,soluong,makho
from biteam.f_sales
where date(ngaychungtu)>=date_sub(current_date(),interval 3 month))
,
dfsc as 
(
select distinct makho,chinhanh chinhanh_sc
from biteam.d_sc_kho_chi_nhanh
)

select t1.masanpham, t2.chinhanh_sc as chinhanh,sum(t1.soluong) soluong,round(sum(t1.soluong)/(90-13),0) as avg_3m
from sales t1
left join dfsc t2 on t1.makho=t2.makho
group by 1,2
"""

SALES = get_bq_df(bsql)

# SALES.to_csv(csv_path+f'{prefix}{name}/'+f'{datenow}_SALES.csv', index=False)


df6 = df5.merge(SALES, how = "left", on=['masanpham','chinhanh'])

df6.soluong.fillna(0, inplace=True)
df6.avg_3m.fillna(0, inplace=True)

# df6['created_date'] = datetime(2022,1,24)
df6['created_date'] = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

assert checkdup(df6,1,['masanpham','chinhanh']).sum() == 0, "MSP & Chi Nhanh khong duoc trung"

In [15]:
df6['created_date'] = datetime(2022,9,28)
df6['inserted_at2'] = datetime(2022,9,28)

In [14]:
# df6.head()

In [9]:
# del_psql = \
# f"""
# DELETE from f_sc_daily_invt where created_date = '{datenow_mns1}'
# """

In [10]:
# commit_psql(del_psql)

In [15]:
# execute_values_insert(df6,'f_sc_daily_invt')

Data inserted using execute_values() successfully..


In [16]:
# datenow_mns1 = (datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d")

In [16]:
del_bsql = \
f"""
DELETE FROM `spatial-vision-343005.biteam.f_sc_daily_invt` where date(created_date) = '{datenow_mns1}'
"""

In [17]:
print(del_bsql)


DELETE FROM `spatial-vision-343005.biteam.f_sc_daily_invt` where date(created_date) = '20220928'



In [18]:
execute_bq_query(del_bsql)

In [19]:
bq_values_insert(df6, "f_sc_daily_invt", 2)